In [ ]:
import krpc
import time
import math
import numpy as np

In [ ]:
conn = krpc.connect(name='PID 1')
vessel = conn.space_center.active_vessel

In [ ]:
import time


class PID:
    """
    This is an implementation of the "Beginner's PID" by Brett Beauregard

    """

    def __init__(self):
        # working variables*/
        self.last_time = time.time()
        self.last_current = 0
        self.output = 0
        self.target = 0
        self.i_out = 0
        self.k_p = 1
        self.k_i = 0
        self.k_d = 0
        self.sample_time = 1  # seconds
        self.out_min = -1
        self.out_max = 1
        self.in_auto = True
        self.output = 0

    def compute(self, current):
        now = time.time()
        if now >= self.last_time + self.sample_time:

            error = self.target - current
            self.i_out += (self.k_i * error)
            if self.i_out > self.out_max:
                self.i_out = self.out_max
            elif self.i_out < self.out_min:
                self.i_out = self.out_min
            d_current = current - self.last_current

            self.output = self.k_p * error + self.i_out - self.k_d * d_current
            if self.output > self.out_max:
                self.output = self.out_max
            elif self.output < self.out_min:
                self.output = self.out_min

            self.last_current = current
            self.last_time = now
        return self.output

    def set_tunings(self, Kp, Ki, Kd):
        self.k_p = Kp
        self.k_i = Ki * self.sample_time
        self.k_d = Kd / self.sample_time

    def set_sample_time(self, new_sample_time):
        if new_sample_time > 0:
            ratio = new_sample_time / self.sample_time
            self.k_i *= ratio
            self.k_d /= ratio
            self.sample_time = new_sample_time

    def set_output_limits(self, min, max):
        self.out_min = min
        self.out_max = max
        if min > max:
            self.out_max = min
            self.out_min = max

        if self.output > self.out_max:
            self.output = self.out_max
        elif self.output < self.out_min:
            self.output = self.out_min

        if self.i_out > self.out_max:
            self.i_out = self.out_max
        elif self.i_out < self.out_min:
            self.i_out = self.out_min

    def set_mode(self, mode):
        self.in_auto = mode
        if mode and not self.in_auto:
            self.i_out = self.output
            if self.i_out > self.out_max:
                self.i_out = self.out_max
            elif self.i_out < self.out_min:
                self.i_out = self.out_min


In [ ]:
"""
This is for controlling the speed of a rover
"""

throttle = PID()
throttle.set_tunings(1, 0.01, 0.01)
throttle.set_sample_time(0.1)
throttle.target = 7.5

standby = False
while vessel.control.gear:
    velocity = vessel.flight(vessel.orbit.body.reference_frame).velocity
    speed = vessel.flight(vessel.orbit.body.reference_frame).speed
    if not standby:
        try_thrott = throttle.compute(speed)
        if try_thrott < 0:
            vessel.control.brakes = True
        else:
            vessel.control.brakes = False
            vessel.control.wheel_throttle = try_thrott
        
        if vessel.resources.amount("ElectricCharge") < 10:
            vessel.control.brakes = True
            vessel.control.wheel_throttle = 0
            standby = True
    if vessel.resources.amount("ElectricCharge") > 105:
        standby = False
    time.sleep(0.1)
vessel.control.brakes = True

In [ ]:
"""
This is some math from an example at http://krpc.github.io/krpc/python
"""
def cross_product(u, v):
    return (u[1]*v[2] - u[2]*v[1],
            u[2]*v[0] - u[0]*v[2],
            u[0]*v[1] - u[1]*v[0])

def dot_product(u, v):
    return u[0]*v[0] + u[1]*v[1] + u[2]*v[2]

def magnitude(v):
    return math.sqrt(dot_product(v, v))

def angle_between_vectors(u, v):
    """ Compute the angle between vector u and v """
    dp = dot_product(u, v)
    if dp == 0:
        return 0
    um = magnitude(u)
    vm = magnitude(v)
    return math.acos(dp / (um*vm)) * (180. / math.pi)

In [ ]:
roll_angle = PID()
roll_angle.set_tunings(0.2, 0.01, 0.01)
roll_angle.set_sample_time(0.1)
roll_angle.target = 0

pitch_angle = PID()
pitch_angle.set_tunings(0.5, 0.01, 0.1)
pitch_angle.set_sample_time(0.1)
pitch_angle.target = 1000
pitch_angle.set_output_limits(-5,10)

elevator = PID()
elevator.set_tunings(0.01, 0.0001, 0.1)
elevator.set_sample_time(0.1)
elevator.target = 0
elevator.set_output_limits(-1,1)

rudder = PID()
rudder.set_tunings(2, 0.001, 0.001)
rudder.set_sample_time(0.1)
rudder.target = 280
rudder.set_output_limits(-60,60)

vessel.control.brakes = False
vessel.control.sas = False
last_alt = vessel.flight().mean_altitude


pitch_angle.target = 3000
# pitch_angle.target = 0
rudder.target = 90


while not vessel.control.sas:
    """
    pitch_angle.target can either control altitude or altitude rate (right now it's set up for altitude control.)
    
    so rudder target will be course, and it will be setting the goals for roll_angle
    roll_angle target will be set by rudder and it will be setting vessel.control.yaw
    elevator target will be set to an altitude rate and it will be setting vessel.control.pitch
    """
    
    vessel_direction = vessel.direction(vessel.surface_reference_frame)
    horizon_direction = (0, vessel_direction[1], vessel_direction[2])
    
    north = (0, 1, 0)
    heading = angle_between_vectors(north, horizon_direction)
    if horizon_direction[2] < 0:
        heading = 360 - heading
    
    up = (1, 0, 0)
    plane_normal = cross_product(vessel_direction, up)
    vessel_up = conn.space_center.transform_direction((0, 0, -1), vessel.reference_frame, vessel.surface_reference_frame)
    roll = angle_between_vectors(vessel_up, plane_normal) - 90
    
    pitch = angle_between_vectors(vessel_direction, horizon_direction)
    
    alt_rate = vessel.flight().mean_altitude - last_alt
    altitude = vessel.flight().mean_altitude
    
    roll_angle.target = rudder.compute(heading)
    vessel.control.roll = roll_angle.compute(roll)
    elevator.target = pitch_angle.compute(altitude)
    # elevator.target = pitch_angle.compute(alt_rate)
    vessel.control.pitch = elevator.compute(pitch)
    
    time.sleep(0.05)
    
vessel.control.roll = 0
vessel.control.pitch = 0
vessel.control.sas = True